### Importing the Libraries

In [31]:
import pandas as pd
import json

In [32]:
df = pd.read_excel("../data/Current Graduate Coordinators.xlsx")

In [33]:
df.columns

Index(['Program Code', 'Concentration', 'Field', 'Department', 'Program',
       'Campus', 'College', 'Activity Indicator', ' Primary Coordinator',
       ' PC - Email', 'Backup Coordinator 1', 'B1 - Email',
       'Backup Coordinator 2', 'B2 - Email', 'Backup Coordinator 3',
       'B3 - Email', 'Backup Coordinator 4', 'B4 - Email', 'Assistant',
       'Asst. - Email', 'Unnamed: 20', 'Unnamed: 21'],
      dtype='object')

In [15]:
standard_columns = [
    "program_code",
    "concentration",
    "field",
    "department",
    "program",
    "campus",
    "college",
    "activity_indicator",
    "primary_coordinator",
    "primary_coordinator_email",
    "backup_coordinator_1",
    "backup_1_email",
    "backup_coordinator_2",
    "backup_2_email",
    "backup_coordinator_3",
    "backup_3_email",
    "backup_coordinator_4",
    "backup_4_email",
    "assistant",
    "assistant_email"
]


### Fix Column nmame inconsistencies

In [16]:
df.columns = [col.strip().lower().replace(' ', '_').replace('-', '_') for col in df.columns]
df = df.loc[:, ~df.columns.str.contains('^unnamed')]

In [17]:
df.columns

Index(['program_code', 'concentration', 'field', 'department', 'program',
       'campus', 'college', 'activity_indicator', 'primary_coordinator',
       'pc___email', 'backup_coordinator_1', 'b1___email',
       'backup_coordinator_2', 'b2___email', 'backup_coordinator_3',
       'b3___email', 'backup_coordinator_4', 'b4___email', 'assistant',
       'asst.___email'],
      dtype='object')

### Convert to Natural language

In [18]:

def excel_row_to_natural_language(row):
    parts = []

    # Basic program info
    if pd.notna(row['program']):
        parts.append(f"The program is '{row['program']}'")
    if pd.notna(row['program_code']):
        parts.append(f"with code '{row['program_code']}'")
    if pd.notna(row['concentration']):
        parts.append(f"under the concentration '{row['concentration']}'")
    if pd.notna(row['field']):
        parts.append(f"in the field of '{row['field']}'")
    if pd.notna(row['department']):
        parts.append(f"offered by the '{row['department']}' department")
    if pd.notna(row['campus']):
        parts.append(f"located at the '{row['campus']}' campus")
    if pd.notna(row['college']):
        parts.append(f"under the '{row['college']}' college")

    # Activity status
    if pd.notna(row['activity_indicator']):
        parts.append(f"The program is currently marked as '{row['activity_indicator']}'.")

    # Coordinator info
    if pd.notna(row['primary_coordinator']) and pd.notna(row['pc___email']):
        parts.append(f"The primary graduate coordinator is {row['primary_coordinator']} ({row['pc___email']}).")

    for i in range(1, 5):
        name_key = f'backup_coordinator_{i}'
        email_key = f'b{i}___email'
        if pd.notna(row.get(name_key)) and pd.notna(row.get(email_key)):
            parts.append(f"Backup Coordinator {i}: {row[name_key]} ({row[email_key]}).")

    if pd.notna(row.get('assistant')) and pd.notna(row.get('asst.___email')):
        parts.append(f"Program Assistant: {row['assistant']} ({row['asst.___email']}).")

    return ' '.join(parts)


In [20]:
df.head()

,program_code,concentration,field,department,program,campus,college,activity_indicator,primary_coordinator,pc___email,backup_coordinator_1,b1___email,backup_coordinator_2,b2___email,backup_coordinator_3,b3___email,backup_coordinator_4,b4___email,assistant,asst.___email
0,ACCT-GC,NaN,ACCX,ACCT,Accounting,Main,CB,Inactive,Sonya Premeaux,sfpremeaux@ualr.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACCT-MACC,NaN,ACCT,ACCT,Accounting-MACC,Main,CB,Inactive,Sonya Premeaux,sfpremeaux@ualr.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AEAX-GC,NaN,AEAX,CHPR,Adult Education: Additional Licensure - GC,Main,BH,Inactive,Jennifer Holtz,jkholtz@ualr.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Alicia Williams,arwilliams@ualr.edu
3,AED-MED,NaN,AED,CHPR,Adult Education - MED,Main,BH,Inactive,Jennifer Holtz,jkholtz@ualr.edu,NaN,NaN,April Chatham-Carpenter,axchathamca@ualr.edu,NaN,NaN,NaN,NaN,Alicia Williams,arwilliams@ualr.edu
4,APCS-MA,NaN,APCS,APCS,Applied Communication Studies - MA,Main,CH,Active,Bailey Blackburn,boblackburn@ualr.edu,Gerald Driskall,gwdriskill@ualr.edu,April Chatham-Carpenter,axchathamca@ualr.edu,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
def convert_excel_to_json_nl(excel_path, output_path):  # nl = natural language
    df = pd.read_excel(excel_path)
    df.columns = [c.strip().lower().replace(' ', '_').replace('-', '_') for c in df.columns]
    df = df.loc[:, ~df.columns.str.contains('^unnamed')]

    with open(output_path, "w") as f:
        for _, row in df.iterrows():
            text = excel_row_to_natural_language(row)
            f.write(json.dumps({"text": text}) + "\n")


In [30]:
convert_excel_to_json_nl("../data/Current Graduate Coordinators.xlsx", "program_grad_coord.jsonl")
